In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-200")
embedding_dim = glove_model.vector_size

[==================================================] 100.0% 252.1/252.1MB downloaded


In [ ]:
print(embedding_dim)

200


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import mutual_info_classif
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources
nltk.download('punkt')

# Load the first Excel file (news-pal.xlsx)
url_pal = "https://github.com/rosenfa/ai/blob/master/news-pal.xlsx?raw=true"
df_pal = pd.read_excel(url_pal)

# Load the second Excel file (news-israel.xlsx)
url_israel = "https://github.com/rosenfa/ai/blob/master/news-israel.xlsx?raw=true"
df_israel = pd.read_excel(url_israel)

df_israel.dropna(inplace=True)
df_pal.dropna(inplace=True)


# Add a 'target' column to both dataframes
df_pal['target'] = False  # Assuming 'false' for news-pal.xlsx
df_israel['target'] = True   # Assuming 'true' for news-israel.xlsx


# Concatenate the two dataframes
df_combined = pd.concat([df_pal, df_israel], ignore_index=True)
df_combined.dropna(inplace=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Assuming df_combined is your DataFrame

# Define a function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in ENGLISH_STOP_WORDS]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to the 'Content' column
df_combined['Content'] = df_combined['Content'].apply(remove_stopwords)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and labels (y)
X = df_combined['Content']
y = df_combined['target']  # Replace 'target_column' with the actual name of your target column

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now you have your training and test sets ready to be used for further processing or modeling


In [ ]:
from keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
X_train= vectorizer(np.array([[s] for s in X_train])).numpy()
X_test= vectorizer(np.array([[s] for s in X_test])).numpy()

In [ ]:
vectorizer.get_vocabulary()[:10]

['',
 '[UNK]',
 'gaza',
 'israeli',
 'israel',
 'hamas',
 'palestinian',
 'war',
 'hospital',
 'israels']

In [ ]:
output = vectorizer([["the gaza fired on the man"]])
output.numpy()[0, :6]

array([ 263,    2,  548, 1176,  263,  177])

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
test = ["gaza", "fired", "on", "israel"]

In [ ]:
vocab_size = len(word_index)
num_tokens = len(voc) + 2
embedding_matrix = np.zeros((num_tokens, embedding_dim))

hits = 0
misses = 0

# Prepare embedding matrix
for word, i in word_index.items():
    if word in glove_model:
      hits += 1
      embedding_matrix[i] = glove_model[word]
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 3479 words (756 misses)


In [ ]:
# print(word_index.items())
# print(print(embedding_matrix[10])) #Note that UNK is like OOV from the Lecture

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="sigmoid")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 200)         847400    
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         128128    
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                      

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"],
)
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
6/6 [==============================] - 5s 687ms/step - loss: 0.6887 - acc: 0.5553 - val_loss: 0.6389 - val_acc: 0.7421
Epoch 2/30
6/6 [==============================] - 2s 395ms/step - loss: 0.5790 - acc: 0.7447 - val_loss: 0.4728 - val_acc: 0.8474
Epoch 3/30
6/6 [==============================] - 2s 403ms/step - loss: 0.4117 - acc: 0.8342 - val_loss: 0.3681 - val_acc: 0.8684
Epoch 4/30
6/6 [==============================] - 2s 379ms/step - loss: 0.2601 - acc: 0.9053 - val_loss: 0.3007 - val_acc: 0.8842
Epoch 5/30
6/6 [==============================] - 2s 376ms/step - loss: 0.1256 - acc: 0.9605 - val_loss: 0.2987 - val_acc: 0.8632
Epoch 6/30
6/6 [==============================] - 4s 626ms/step - loss: 0.0688 - acc: 0.9803 - val_loss: 0.3224 - val_acc: 0.8842
Epoch 7/30
6/6 [==============================] - 2s 399ms/step - loss: 0.0340 - acc: 0.9934 - val_loss: 0.2923 - val_acc: 0.8947
Epoch 8/30
6/6 [==============================] - 2s 401ms/step - loss: 0.0117 - acc: 1.00